In [1]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import segmentation_models as sm

2022-11-06 10:49:08.392957: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-06 10:49:08.475043: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-06 10:49:08.493529: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Segmentation Models: using `keras` framework.


In [2]:
ROOT_DIR = os.path.abspath(os.getcwd())
DATASETS_DIR = os.path.join(ROOT_DIR, "datasets")
ICH_420_DATASET = os.path.join(DATASETS_DIR, "ICH_420")
ICH_420_DATASET_TFRECORDS = os.path.join(ICH_420_DATASET, "TFRecords")
ICH_420_DATASET_TFRECORDS_TRAIN = os.path.join(ICH_420_DATASET_TFRECORDS, "train")

In [3]:
def split_train_test(tfrecords_pattern, test_rate=0.2, buffer_size=10000):
    filenames = tf.io.gfile.glob(tfrecords_pattern)
    random.shuffle(filenames)
    split_idx = int(test_rate * len(filenames))
    

    return filenames[split_idx:], filenames[:split_idx]

In [4]:
train_dataset_filepaths, test_dataset_filepaths = split_train_test(os.path.join(ICH_420_DATASET_TFRECORDS_TRAIN, "*.tfrecord"))
print(f"Train: {len(train_dataset_filepaths)}")
print(f"Test: {len(test_dataset_filepaths)}")

Train: 10
Test: 2


In [5]:
def decode_image(image, bit=12):
    image = tf.io.decode_png(image, 1, dtype=tf.dtypes.uint16)
    image = tf.expand_dims(image, axis=-1)
    image = tf.cast(image, tf.dtypes.float32) / (2 ** bit)
    
    return image

In [6]:
def read_tfrecord(example):
    features_description = {
        "filename": tf.io.FixedLenFeature([], tf.string),
        "number": tf.io.FixedLenFeature([], tf.int64),
        "sample": tf.io.FixedLenFeature([], tf.int64),
        "image_raw": tf.io.FixedLenFeature([], tf.string),
        "mask_raw": tf.io.FixedLenFeature([], tf.string)
    }
    
    example = tf.io.parse_single_example(example, features_description, name="nii")
    image = decode_image(example["image_raw"])
    mask = decode_image(example["mask_raw"])

    return image, mask

In [7]:
def load_dataset(filenames):
    dataset = tf.data.TFRecordDataset(
        filenames,
        compression_type="GZIP"
    )
    dataset = dataset.shuffle(2048, reshuffle_each_iteration=False)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=tf.data.AUTOTUNE)
    
    return dataset

In [8]:
def get_dataset(filenames):
    dataset = load_dataset(filenames)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(8)
    dataset = dataset.repeat()
    
    return dataset

In [9]:
train_dataset = get_dataset(train_dataset_filepaths)
test_dataset = get_dataset(test_dataset_filepaths)

train, test = iter(train_dataset).next()

2022-11-06 10:49:09.733230: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-06 10:49:09.734917: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-06 10:49:09.734992: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-06 10:49:09.735452: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

In [10]:
preprocess_input = sm.get_preprocessing("resnet50")
train_dataset = preprocess_input(train_dataset)

In [11]:
model = sm.Unet("resnet50", encoder_weights=None, input_shape=(None, None, 1))
# tf.keras.utils.plot_model(model, show_dtype=True)

In [12]:
model.compile(
    'Adam',
    loss=sm.losses.dice_loss,
    metrics=[sm.metrics.iou_score, sm.metrics.f1_score],
)

In [13]:
model.fit(train_dataset,
          epochs=2,
          steps_per_epoch=1000,
          verbose=2
         )

Epoch 1/2


2022-11-06 10:49:15.485422: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-11-06 10:49:16.345074: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1000/1000 - 251s - loss: 0.9997 - iou_score: 1.6716e-04 - f1-score: 2.6127e-04 - 251s/epoch - 251ms/step
Epoch 2/2
1000/1000 - 244s - loss: 0.9475 - iou_score: 0.0525 - f1-score: 0.0525 - 244s/epoch - 244ms/step
